In [ ]:
%matplotlib inline
import afgiftspligtig_værdi as av
import matplotlib.patches as mpatches
import logit
import clogit_estimation as cle
import pandas as pd
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt
import requests as rq
from io import BytesIO

In [ ]:
path = "https://raw.githubusercontent.com/Jona327a/Bachelor-projekt/main/Data/choice_data_subset.xlsx"

In [ ]:
def data_set(path):
    data = rq.get(path).content
    choice_data_subset = pd.read_excel(BytesIO(data))
    return choice_data_subset

In [ ]:
def theta_hat():
    choice_data_subset = data_set(path)
    
    choice_data_subset['Prices (2015-DKK)'] = np.log(choice_data_subset['Prices (2015-DKK)'] / 100000)
    choice_data_subset['Weight (kg)'] = choice_data_subset['Weight (kg)'] / 1000
    choice_data_subset['Engine effect (kW)'] = choice_data_subset['Engine effect (kW)'] / 100
    
    variables = ['Prices (2015-DKK)', 'Weight (kg)', 'Engine effect (kW)', 'Size (m3)', 'Cost/km (DKK)']
    N = choice_data_subset.Year.nunique()
    J = choice_data_subset[['Year', 'key']].groupby('Year').count().min().tolist()[0]
    x, y, x_vars = cle.get_x_y(dataset = choice_data_subset, N = N, J = J,  dummyvar = 'Fuel', attributes = variables)
    
    result = cle.clogt_estimation(x = x, y = y, x_vars = x_vars)
    theta_hat = result['theta_hat']
    return theta_hat

In [ ]:
def new_prices(rate_1, rate_2, rate_3, reg_beløb_1, reg_beløb_2, bundfradrag, el_bundfradrag_1, el_fradrag_2, indfasning):
    choice_data_subset = data_set(path)
    
    new_prices = []
    for i in range(0, choice_data_subset.shape[0]):
        av_i = choice_data_subset.loc[i, 'afgiftspligtig_værdi']
        fuel_i = choice_data_subset.loc[i, 'Fuel']
        if fuel_i == 'El':
            new_price = av.pris_efter_reg_afgift_el(afgiftspligtig_værdi = av_i, bundfradrag = bundfradrag, ekstra_bundfradrag = el_bundfradrag_1, indfasning = indfasning, 
                                                    reg_beloeb1 = reg_beløb_1, reg_beloeb2 = reg_beløb_2, el_fradrag = el_fradrag_2,
                                                    part_1 = rate_1, part_2 = rate_2, part_3 = rate_3)
            new_prices.append(new_price)
        elif fuel_i == 'Benzin':
            new_price = av.pris_efter_reg_afgift_ben(afgiftspligtig_værdi = av_i, bundfradrag = bundfradrag, 
                                                    reg_beloeb1 = reg_beløb_1, reg_beloeb2 = reg_beløb_2, 
                                                    part_1 = rate_1, part_2 = rate_2, part_3 = rate_3)
            new_prices.append(new_price)
        else:
            new_price = av.pris_efter_reg_afgift_die(afgiftspligtig_værdi = av_i, bundfradrag = bundfradrag, 
                                                    reg_beloeb1 = reg_beløb_1, reg_beloeb2 = reg_beløb_2,
                                                    part_1 = rate_1, part_2 = rate_2, part_3 = rate_3)
            new_prices.append(new_price)
    return new_prices

In [ ]:
def interactive_cp(rate_1 = 0.25, rate_2 = 0.85, rate_3 = 1.5, reg_beløb_1 = 65000, reg_beløb_2 = 202200, bundfradrag = 21700, el_bundfradrag_1 = 170000, el_fradrag_2 = 1700, indfasning = 0.40):
    
    choice_data_subset = data_set(path)
    
    choice_data_subset['Prices (2015-DKK)'] = np.log(choice_data_subset['Prices (2015-DKK)'] / 100000)
    choice_data_subset['Weight (kg)'] = choice_data_subset['Weight (kg)'] / 1000
    choice_data_subset['Engine effect (kW)'] = choice_data_subset['Engine effect (kW)'] / 100

    variables = ['Prices (2015-DKK)', 'Weight (kg)', 'Engine effect (kW)', 'Size (m3)', 'Cost/km (DKK)']
    N = choice_data_subset.Year.nunique()
    J = choice_data_subset[['Year', 'key']].groupby('Year').count().min().tolist()[0]
    x, y, x_vars = cle.get_x_y(dataset = choice_data_subset, N = N, J = J,  dummyvar = 'Fuel', attributes = variables)

    result = cle.clogt_estimation(x = x, y = y, x_vars = x_vars)
    theta_hat = result['theta_hat']
    
    choice_data_subset = choice_data_subset.assign(new_prices = new_prices(rate_1, rate_2, rate_3, reg_beløb_1, reg_beløb_2, bundfradrag, el_bundfradrag_1, el_fradrag_2, indfasning))
    choice_data_subset['new_prices'] = np.log(choice_data_subset['new_prices'] / 100000)

    variables_v2 = ['new_prices', 'Weight (kg)', 'Engine effect (kW)', 'Size (m3)', 'Cost/km (DKK)']
    x_v2, y_v2, x_vars_v2 = cle.get_x_y(dataset = choice_data_subset, N = N, J = J,  dummyvar = 'Fuel', attributes = variables_v2)
    
    cp = logit.choice_prob(theta_hat, x_v2)
    cp = cp.reshape(N*J,)
    choice_data_subset = choice_data_subset.assign(cp = cp)
    #print("\n Changes in 'new_prices':\n", choice_data_subset['new_prices'])
    #print("\n Changes in 'CPs':\n", cp)

    cp_data = choice_data_subset[['Year', 'cp', 'Fuel']].set_index(['Fuel'])

    cp_el_data = cp_data.loc['El']
    cp_el_sum = cp_el_data.groupby('Year')['cp'].sum()

    cp_benzin_data = cp_data.loc['Benzin']
    cp_benzin_sum = cp_benzin_data.groupby('Year')['cp'].sum()
    
    cp_diesel_data = cp_data.loc['Diesel']
    cp_diesel_sum = cp_diesel_data.groupby('Year')['cp'].sum()

    print("\nSummen af CPs i 2011 for hver biltype:")
    print("Elbiler =", round(cp_el_sum[2011], 6))
    print("Benzinbiler =", round(cp_benzin_sum[2011], 6))
    print("Dieselbiler =", round(cp_diesel_sum[2011], 6))
    
    print("\nSummen af CPs i 2020 for hver biltype:")
    print("Elbiler =", round(cp_el_sum[2020], 6))
    print("Benzinbiler =", round(cp_benzin_sum[2020], 6))
    print("Dieselbiler =", round(cp_diesel_sum[2020], 6))
        
    fig = plt.figure()
    ax1 = fig.add_subplot(1, 1, 1)

    ax1.set_xlabel('År')
    ax1.set_ylabel('Elbiler CPs', color = 'black') 
    line1, = ax1.plot(cp_el_data['Year'].drop_duplicates(), cp_el_sum, color = 'g')
    ax1.set_ylim([0.0, 0.05])
    ax1.grid()

    ax2 = ax1.twinx()
    ax2.set_ylabel('Benzin- og dieselbiler CPs', color = 'black') 
    line2, = ax2.plot(cp_benzin_data['Year'].drop_duplicates(), cp_benzin_sum, color = 'r')
    ax2.set_ylim([0.0, 1.0])

    line3, = plt.plot(cp_diesel_data['Year'].drop_duplicates(), cp_diesel_sum, color = 'b')

    el_label = mpatches.Patch(color = 'g', label = 'El')
    benzin_label = mpatches.Patch(color = 'r', label = 'Benzin')
    diesel_label = mpatches.Patch(color = 'b', label = 'Diesel')
    ax1.legend(handles = [el_label, benzin_label, diesel_label])
    plt.title("Efterspørgslen ved ændring i registreringsafgiften")

    line1.set_ydata(cp_el_sum)
    line2.set_ydata(cp_benzin_sum)
    line3.set_ydata(cp_diesel_sum)
    fig.canvas.draw_idle()

#interact(interactive_cp, rate_1 = (0.0, 0.5, 0.02), rate_2 = (0.0, 1.7, 0.02), rate_3 = (0.0, 3.0, 0.02), reg_beløb_1 = (0.0, 77100.0, 500.0), reg_beløb_2 = (0.0, 240000.0, 500.0), bundfradrag = (0.0, 25700.0, 250.0), el_bundfradrag_1 = (147500.0, 170000.0, 500.0), el_fradrag_2 = (0.0, 1700.0, 250.0), indfasning = (0.0, 1.0, 0.1));


In [ ]:
# Getting the versions of the used packages
"""
%reload_ext watermark

%watermark -v -m -p matplotlib,pandas,ipywidgets,numpy,requests,io,import_ipynb,interactive_plot_code,watermark

print(" ")
%watermark -u -n -t -z
"""